<a href="https://colab.research.google.com/github/Gordon-4389/Face-Anonymisation-Benchmarking/blob/main/Exp_3_3_DeepPrivacy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# @ title Check CUDA Version
!pip install utils
!pip install --upgrade tensorrt --quiet
# !pip install --upgrade tensorflow[and-cuda] --quiet
# !pip install --upgrade torch
!nvidia-smi
# !pip uninstall opencv-python-headless
# !pip install opencv-python-headless

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=76b7ff88d9d7d52412c1eb7bd4465f0adb9b43b937cf79e15bd56b9fadbb8bbf
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils
  Preparing metadata (setup.py) ... done
Sun Mar 31 07:43:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+===================

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("CUDA version:")
!nvcc --version
# print("cuDNN version:")
# !apt list --installed | grep cudnn

import torch
print("Torch version:", torch.__version__)
print("CUDA version:")
# !dpkg -L libcudnn8
# !dpkg -L libcudnn8-dev
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
# !cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

TensorFlow version: 2.15.0
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Torch version: 2.2.2+cu121
CUDA version:


In [ ]:
# @title Check TensorRT installation
# !dpkg -l | grep nvinfer
!pip show tensorrt

Name: tensorrt
Version: 8.6.1.post1
Summary: A high performance deep learning inference library
Home-page: https://developer.nvidia.com/tensorrt
Author: NVIDIA Corporation
Author-email: 
License: Proprietary
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


# Setup

In [ ]:
# @title Download DP2
!rm -rf deep_privacy2
!git clone https://github.com/hukkelas/deep_privacy2/

Cloning into 'deep_privacy2'...
remote: Enumerating objects: 611, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 611 (delta 60), reused 125 (delta 48), pack-reused 434
Receiving objects: 100% (611/611), 13.73 MiB | 29.85 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [ ]:
# @title Code to replace setup dependencies

# function to replace strings in the code
def replace_string_txt_file(to_sub_string: str, replace_string: str, file_path: str):
    with open(file_path, 'r') as file:
        filedata = file.read()

    filedata = filedata.replace(to_sub_string, replace_string) # Reokace String
    with open(file_path, 'w') as file:
        file.write(filedata)
    return

# Fix 'scipy==1.7.1' -> 'scipy==1.8.0' in setup.py
# Should be fine to do so, seeing that there are no changes: https://github.com/statsmodels/statsmodels/compare/v0.14.0...v0.13.5?diff=unified&w=
replace_string_txt_file("\"scipy==1.7.1\",","\"scipy\",", 'deep_privacy2/setup.py')
replace_string_txt_file("\"setuptools==59.5.0\",","\"setuptools\",", 'deep_privacy2/setup.py')

# Specify Pillow = 8.4.0
replace_string_txt_file("pillow==8.3.1", "pillow", 'deep_privacy2/setup.py')

# Change to Latest Release of Detectron2


In [ ]:
# @title Install DP2
%cd deep_privacy2
! pip install -e . --quiet
%cd ..

# Import Data

In [ ]:
# # @title Import Zipped Data From Google Drive
# import gdown, os
# import shutil
# from google.colab import drive

# # os.makedirs("/content/DeepPrivacy2_Data", exist_ok=True)

# # gdown.download_folder('https://drive.google.com/drive/u/0/folders/1FaLmr59_RGAeAIgAy0B_Ede2XbxbeOPd', quiet=True)
# # Unpack Data
# # zipped_path = '/content/DeepPrivacy2_Data/fdf256_DeepPrivacy2.zip'
# # unzipped_path = '/content/DeepPrivacy2_Data/'
# # shutil.unpack_archive(zipped_path, unzipped_path)
# # print(len(os.listdir('/content/DeepPrivacy2_Data/fdf256_DeepPrivacy2')))

# # os.makedirs('/content/fdf256_deepprivacy2_output',  exist_ok=True)
# # shutil.unpack_archive('/content/DeepPrivacy2_Data/fdf256_deepprivacy2_output.zip', '/content/fdf256_deepprivacy2_output')
# # print(len(os.listdir('/content/fdf256_deepprivacy2_output')))

# # Test_dataset 20K
# # gdown.download_folder('https://drive.google.com/drive/folders/1R0h5GAAh8AtCnA_2nnRiryPySwyWvefT')

# # Mount Drive
# drive.mount('/content/drive')
# unzipped_path = '/content/Testing_Dataset_20k/'
# shutil.unpack_archive('/content/drive/MyDrive/Testing_Dataset_20k/test_dataset_20k_fdf256.zip', unzipped_path)
# shutil.unpack_archive('/content/drive/MyDrive/Testing_Dataset_20k/test_dataset_20k_CelebA-HQ.zip', unzipped_path)

In [ ]:
# @title Importing Testing_Dataset Folder from Drive
import gdown, os
import shutil
from google.colab import drive

# !rm -rf /content/Testing_Dataset_20k

# Mount Drive
# drive.mount('/content/drive', force_remount=False)

shutil.unpack_archive('/content/drive/MyDrive/Testing_Dataset_20k/test_dataset_20k_fdf256.zip', '/content/Testing_Dataset_20k/test_dataset_20k_fdf256')
shutil.unpack_archive('/content/drive/MyDrive/Testing_Dataset_20k/test_dataset_20k_CelebA-HQ.zip', '/content/Testing_Dataset_20k/test_dataset_20k_CelebA-HQ')

# Anonymisation

In [ ]:
# @title Set Dataset
dataset = "CelebA-HQ" # @param ["fdf256", "CelebA-HQ"]
print(f"Testing folder_path: /content/Testing_Dataset_20k/test_dataset_20k_{dataset}")
print(f"Output Folder: /content/{dataset}_DeepPrivacy2")

Testing folder_path: /content/Testing_Dataset_20k/test_dataset_20k_CelebA-HQ
Output Folder: /content/CelebA-HQ_DeepPrivacy2


In [ ]:
# Copying Partial Results if Any
print("Copying Partial Results")
!cp -r /content/drive/MyDrive/Testing_Dataset_20k/{dataset}_DeepPrivacy2  /content/Testing_Dataset_20k/{dataset}_DeepPrivacy2

In [ ]:
# @title Anonymise Directory with 256x256 Model - Not Working
# !python3 anonymize.py configs/anonymizers/FB_cse.py -i path_to_image.png

# Make Output Folder
# !rm -rf fdf256_deepprivacy2_output
# !mkdir fdf256_deepprivacy2_output

# Anonymise
%cd /content/deep_privacy2
! python3 anonymize.py configs/anonymizers/face.py -i ../Testing_Dataset_20k/test_dataset_20k_{dataset} --output_path ../Testing_Dataset_20k/{dataset}_DeepPrivacy2

In [ ]:
# @title Face Anonymise - Single Image
%cd /content/deep_privacy2
! python3 anonymize.py configs/anonymizers/face.py -i ../DeepPrivacy2_Data/fdf256_DeepPrivacy2/002001.png --output_path ../002001.png


In [ ]:
# @title Face Anonymise - Multiple Images 'Manual' - Step 1
%cd ~/../content/deep_privacy2

import glob
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
count = 0

input_images = glob.glob(f'/content/Testing_Dataset_20k/test_dataset_20k_{dataset}/*.png')
output_folder_path =f'/content/Testing_Dataset_20k/{dataset}_DeepPrivacy2'
print(len(input_images))
check_output = glob.glob(output_folder_path+'/*.png')
print(len(check_output))

/content/deep_privacy2
20000
2550


In [ ]:
# @title Manual - Step 2
# Anonymise
for image in input_images:
  # clear_output(wait=True) # Clear output
  image_num = image.split('/')[-1]
  o_path = output_folder_path + '/' + image_num
  print(image_num)
  if o_path not in check_output:
    ! python3 anonymize.py configs/anonymizers/face.py -i {image} --output_path {o_path}
    count += 1
    print(count)

Streaming output truncated to the last 5000 lines.
426
233934.png
2024-03-31 11:34:58.611812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 11:34:58.611859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 11:34:58.613205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-31 11:34:59.829813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib c

# Save Results

In [ ]:
# @title Upload to Drive
%cd ~/../content/
path = f'/content/Testing_Dataset_20k/{dataset}_DeepPrivacy2'
print(len(os.listdir(path)))
# !cp -r /content/Testing_Dataset_20k/test_dataset_20k_CelebA-HQ /content/drive/MyDrive/Testing_Dataset_20k/
# !cp -r /content/Testing_Dataset_20k/test_dataset_20k_fdf256 /content/drive/MyDrive/Testing_Dataset_20k/
!cp -r {path} /content/drive/MyDrive/Testing_Dataset_20k/

/content
20000


In [ ]:
# @title Save Results in a zip file and download:
import os, shutil
%cd ~/../content/
path = '/content/fdf256_DeepPrivacy2'
print(len(os.listdir(path)))
# if len(os.listdir(path)) == 50:
shutil.make_archive(path, 'zip', path)
# Download Files
from google.colab import files
files.download(f'{path}.zip')

/content
2093


'/content/fdf256_DeepPrivacy2.zip'

In [ ]:
# @title Simple code to tell which Image is Anonymised
import glob, os
input_images = glob.glob('/content/DeepPrivacy2_Data/fdf256_DeepPrivacy2/*.png')
for i in range(len(input_images)):
  input_num = input_images[i].split('/')[-1]
  if input_num in os.listdir('/content/fdf256_deepprivacy2_output'):
    print(input_num, 'found')
  else:
    print(input_num, 'not found')